In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from scipy import stats
from sklearn.model_selection import KFold
import math

In [4]:
# rng = np.random.RandomState(42)
# clf = IsolationForest(max_samples=n_samples, random_state=rng, contamination=outliers_fraction)
# clf.fit(X_train)
df = pd.read_csv("test_train.csv").drop(columns = ['Unnamed: 0'])

In [5]:
df

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_is_S2,flag_is_S0,flag_is_S3,flag_is_RSTO,flag_is_RSTR,flag_is_RSTOS0,flag_is_OTH,flag_is_SH,normal,test
0,0,181,5450,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
1,0,239,486,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
2,0,235,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
3,0,219,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
4,0,217,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,310,1881,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
494017,0,282,2286,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
494018,0,203,1200,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0
494019,0,291,1200,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,0.0


In [6]:
kf = KFold(n_splits=10, random_state=57, shuffle=True)
for i in range(5):
    #for each category of attack, split the data into the required test and train sections
    dftrain = df.loc[df['test'] != i]
    dfxtest = df.loc[df['test'] == i].iloc[:,:-2]
    dfytest = df.loc[df['test'] == i].iloc[:,-2]
    for split in kf.split(dftrain):
        #on the training data, do a 10-fold 
        train = dftrain.iloc[split[0]]
        test = dftrain.iloc[split[1]]
        xtrain = train.iloc[:,:-2]
        xtest = test.iloc[:,:-2]
        ytrain = train.iloc[:,-2]
        ytest = test.iloc[:,-2]
        #model for training data goes here
        #select model with highest score
    #using the selected model for each category test the model against dfxtest and dfytest

In [7]:
# outliers_fraction = xtrain
xtrain.shape[0]

440923

In [8]:
m = xtrain.shape[0]
outliers_fraction = 2*math.log(m-1)+0.5772156649

In [9]:
outliers_fraction

26.570462200974415

In [10]:
clf = IsolationForest()
clf.fit(xtrain)
scores_pred = clf.decision_function(xtrain)
threshold = stats.scoreatpercentile(scores_pred, outliers_fraction)  
# scores_pred


In [11]:
# print("1",xtrain[y_pred_train==1])
# print("-1",xtrain[y_pred_train==-1])

In [12]:
scores_pred

array([0.03095543, 0.0368036 , 0.03474043, ..., 0.01163937, 0.03505756,
       0.04241385])

In [13]:
scores_pred.describe()

AttributeError: 'numpy.ndarray' object has no attribute 'describe'

In [32]:
# plot the line, the samples, and the nearest vectors to the plane
xx, yy = np.meshgrid(np.linspace(-7, 7, 50), np.linspace(-7, 7, 50))
Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
# Z = Z.reshape(xx.shape)

D:\Users\Cryst\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


ValueError: X has 2 features, but IsolationForest is expecting 118 features as input.

In [ ]:
plt.title("IsolationForest")
# plt.contourf(xx, yy, Z, cmap=plt.cm.Blues_r)
plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), threshold, 7), cmap=plt.cm.Blues_r)  #绘制异常点区域，值从最小的到阈值的那部分
a = plt.contour(xx, yy, Z, levels=[threshold], linewidths=2, colors='red')  #绘制异常点区域和正常点区域的边界
plt.contourf(xx, yy, Z, levels=[threshold, Z.max()], colors='palevioletred')  #绘制正常点区域，值从阈值到最大的那部分
 
b = plt.scatter(X_train[:-n_outliers, 0], X_train[:-n_outliers, 1], c='white',
                    s=20, edgecolor='k')
c = plt.scatter(X_train[-n_outliers:, 0], X_train[-n_outliers:, 1], c='black',
                    s=20, edgecolor='k')
plt.axis('tight')
plt.xlim((-7, 7))
plt.ylim((-7, 7))
plt.legend([a.collections[0], b, c],
           ['learned decision function', 'true inliers', 'true outliers'],
           loc="upper left")
plt.show()

In [35]:
from pyod.models.iforest import IForest
from pyod.utils.data import generate_data

from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
import pandas as pd

In [34]:
pip install pyod

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for pyod, since package 'wheel' is not installed.
    Running setup.py install for pyod: started
    Running setup.py install for pyod: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


In [38]:
clf_name = 'IForest'
clf = IForest()
clf.fit(xtrain)

# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(xtest)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(xtest)  # outlier scores

# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, ytrain, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, ytest, y_test_scores)

# visualize the results
# visualize(clf_name, xtrain, ytrain, xtest, ytest, y_train_pred,
#           y_test_pred, show_figure=True, save_figure=False)

D:\Users\Cryst\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:434: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
D:\Users\Cryst\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:434: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(



On Training Data:
IForest ROC:0.8091, precision @ rank n:0.4779

On Test Data:
IForest ROC:0.8085, precision @ rank n:0.4753
